In [82]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.layers import *
from keras.models import *
from keras.utils import *
from sklearn.preprocessing import *
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import random
import re


In [83]:
file = 'lastofus.txt'
text = ''
array=[]
with open(file) as f:
    lines = f.readlines()
#     for i in lines:
#         print(i)    
    text = text.join([l for l in lines if re.match(r'^[A-Z].*:', l)])

# for t in text.split('\n'):
#     print(t)

# for t in text.split('\n'):
#     array.append(t)
# array

for t in text.split('\n')[:1000]:
    array.append(t)
# array

In [84]:
token = Tokenizer(lower=False, filters='.,?;\'\"-')
token.fit_on_texts(array)

In [85]:
# token.word_index
# len(token.word_index) # 1384

seq = token.texts_to_sequences(array)
# seq

# (sample, timestamp, feature) timestamp 기준으로 문장 조절하기



In [86]:
seq = pad_sequences(seq, maxlen=10)
seq

array([[ 130,  626,    8, ...,  627,  104,    3],
       [   0,    0,    0, ...,    0,   17,   58],
       [   0,    0,    0, ...,    0,    1,  628],
       ...,
       [  57, 1382,   22, ...,   21,   55,  418],
       [   0,    0,    0, ...,    0,    7,  328],
       [   0,    0,    0, ...,    1, 1384,    9]], dtype=int32)

In [87]:
seq.shape # (1000, 10)
X = seq
X

array([[ 130,  626,    8, ...,  627,  104,    3],
       [   0,    0,    0, ...,    0,   17,   58],
       [   0,    0,    0, ...,    0,    1,  628],
       ...,
       [  57, 1382,   22, ...,   21,   55,  418],
       [   0,    0,    0, ...,    0,    7,  328],
       [   0,    0,    0, ...,    1, 1384,    9]], dtype=int32)

In [88]:
Y = np.vstack((X[1:], X[0])) # 한 칸씩 올리고 맨 위에거는 맨 아래로 내려...
Y

array([[   0,    0,    0, ...,    0,   17,   58],
       [   0,    0,    0, ...,    0,    1,  628],
       [   0,    0,    0, ...,  111,  266,  331],
       ...,
       [   0,    0,    0, ...,    0,    7,  328],
       [   0,    0,    0, ...,    1, 1384,    9],
       [ 130,  626,    8, ...,  627,  104,    3]], dtype=int32)

In [89]:
X = X.reshape(-1, 10, 1)
Y = Y.reshape(-1, 10, 1)
X.shape # (1000, 10, 1)

(1000, 10, 1)

In [90]:
Y = to_categorical(Y)
Y.shape # (1000, 10, 1385) 1385는 단어의 갯수... 1384 + 1(0부터 시작이니까...)

(1000, 10, 1385)

In [91]:
'''
-> -> ->

Bidrectional을 적용하면 좀 더 효율적인 학습?
-> -> ->
<- <- <-
'''

model = Sequential()
model.add(Bidirectional(SimpleRNN(128, return_sequences=True), input_shape=(10, 1)))
model.add(Bidirectional(SimpleRNN(128, return_sequences=True)))
model.add(Dense(1385, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_7 (Bidirectio  (None, 10, 256)          33280     
 nal)                                                            
                                                                 
 bidirectional_8 (Bidirectio  (None, 10, 256)          98560     
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 10, 1385)          355945    
                                                                 
Total params: 487,785
Trainable params: 487,785
Non-trainable params: 0
_________________________________________________________________


In [92]:
model.fit(X, Y, epochs=1, batch_size=1)
model.save('ddd')

1000/1000 [==============================] - 13s 10ms/step - loss: 5.0057 - accuracy: 0.3235
INFO:tensorflow:Assets written to: ddd/assets


In [93]:
model = load_model('ddd')

In [106]:
pred = model.predict(X[:1])
# pred.shape

1/1 [==============================] - 0s 24ms/step


In [107]:
# 역순으로 해주세요...
pred = np.argmax(pred, axis=2) # 숫자로...
pred
# pred.shape # 1, 10

array([[  0,   0,   0,   0,   0,   0,   0,  39, 302, 685]])

In [102]:
idx_word = {}
for w in token.word_index:
    idx_word[token.word_index[w]] = w
# idx_word # token의 key와 value 바꾸기

In [108]:
temp = ''
for l in pred:
    for w in l:
#         print(w)
        if w != 0:
            temp += idx_word[w]
            temp += ' '
print(temp)

for low minds 


In [104]:
# 여기부터 좀 이해하기 어렵네...
for i in range(10):
    temp = ''
    ran = random.randrange(0, len(X))
    pred = model.predict(np.expand_dims(X[ran], axis=0))
    
    pred = np.argmax(pred, axis=2)
    
    for l in pred:
        for w in l:
            if w != 0:
                temp += idx_word[w]
                temp += ' '
        temp += '\n'
    print(array[ran])     # 입력
    print('\n\n\n')
    print(temp) # 출력
    
    

1/1 [==============================] - 0s 24ms/step
Joel: Alright, watch your head. Hurry. Go, go, go. Shit...sonofa...




the low time 

1/1 [==============================] - 0s 23ms/step
Joel: They ain't spotted us yet, let's go around.




a time 

1/1 [==============================] - 0s 28ms/step
Tess: I was on my way back here, and I got jumped by these two assholes,




of for time 

1/1 [==============================] - 0s 21ms/step
Joel: Don't you think it's a bit strange that they're askin' us to do their




a time 

1/1 [==============================] - 0s 25ms/step
Ellie: There's so many of 'em out there. How are we supposed to get past 'em?




a for low 

1/1 [==============================] - 0s 29ms/step
Tess: I think I can handle it.




a for minds 

1/1 [==============================] - 0s 25ms/step
Tommy: Oh, this is bad. Everyone and their mother had the same damn idea.




need for time 

1/1 [==============================] - 0s 26ms/step
Joel: Shit, I see